# Tutorial: Model Training / Inference based on MixedDatasets

# 0 Configure ENVS

In [ ]:
!git clone https://github.com/rwightman/pytorch-image-models.git

%cd pytorch-image-models/
!python -m pip --no-cache-dir install -r requirements.txt
!python setup.py develop

# 1 MixedDataset: Querying, Training and Testing

## 1.1 Query a MixedDataset

In [ ]:
# install our vision utils
#!python -m pip install git+https://github.com/cqels/vision.git --force


In [ ]:
# import SemkgAPI
#import json
#from vision_utils import semkg_api, data

# # query from string

# # query from file

In [ ]:
# import json
# import os
# from os.path import join as opj
# from utils import dataset_split, check_instances_categories, check_download_images


# query_string='''#Give me the images containing person and cat
# prefix cv:<http://vision.semkg.org/onto/v0.1/>
# SELECT DISTINCT ?image
# WHERE {
#     ?ann1 a cv:Annotation.
#     ?ann1 cv:isAnnotationOfImage ?image.
#     ?ann1 cv:hasAnnotatedObject ?obj1.
#     ?obj1 cv:hasLabel "person".
#     ?ann2 a cv:Annotation.
#     ?ann2 cv:isAnnotationOfImage ?image.
#     ?ann2 cv:hasAnnotatedObject ?obj2.
#     ?obj2 cv:hasLabel "cat".
#     ?image cv:hasLocalPath ?localPath.
# }
# LIMIT 20'''

# result=semkg_api.query(query_string)

# ROOT_PATH = os.path.abspath('./')
# json_f_name = 'test_query_api_image.json'
# path_to_anno_mixedDatasets = opj(ROOT_PATH, 'testData/mixedDatasets/imagenette2_tiny/meta/')
# path_to_images_mixedDatasets = opj(ROOT_PATH, 'testData/mixedDatasets/imagenette2_tiny/')
# os.makedirs(path_to_anno_mixedDatasets, exist_ok=True)
# path_to_anno = opj(path_to_anno_mixedDatasets, json_f_name)

# with open(path_to_anno, "w") as f:
#     json.dump(result,f)

# check_download_images(result["images"])
# categories = [category['name'] for category in result['categories']]
# number_of_categories = len(categories)

## 1.1 Download ImageNette

In [ ]:
!wget -P ./data/ https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xf ./data/imagenette2-320.tgz -C ./data/ && rm ./data/imagenette2-320.tgz

## 1.2 Set dirs && other params

In [ ]:
import os

data_dir = './data/imagenette2-320/'
test_images = './data/imagenette2-320/val/'
output_training_dir = './output/train/'
output_test_dir = './output/test/'
num_classes = len(os.listdir(test_images))
used_model = 'resnet18'

## 1.3 Training on the MixedDataset

In [ ]:
%run train.py {data_dir} \
--output {output_training_dir} \
--model {used_model} \
--sched cosine \
--epochs 10 \
--warmup-epochs 5 \
--lr 0.4 \
--reprob 0.5 \
--remode pixel \
--checkpoint-hist 10 \
--num-classes {num_classes} \
--batch-size 256

## 1.4 Verify the checkpoint file.

In [ ]:
all_existed_dirs = [os.path.join(output_training_dir,sub_dir) for sub_dir in os.listdir(output_training_dir) if os.path.isdir(os.path.join(output_training_dir,sub_dir))]
latest_output_dir = max(all_existed_dirs, key=os.path.getmtime)
checkpoint_file = os.path.join(latest_output_dir, "model_best.pth.tar")
assert os.path.isfile(checkpoint_file), '{} not exist'.format(checkpoint_file)
checkpoint_file = os.path.abspath(checkpoint_file)

## 1.5 Testing on the MixedDataset

In [ ]:
%run validate.py {test_images} \
--model {used_model} \
--num-classes {num_classes} \
--checkpoint {checkpoint_file}